In [1]:
import xarray as xr

In [2]:
papa_data = xr.load_dataset(
    "../stations/papa/2_processed/papa_zooplankton.zarr", engine="zarr"
)
papa_data

<xarray.Dataset> Size: 297MB
Dimensions:           (time: 623, is_day: 2, latitude: 12, longitude: 31,
                       depth: 8)
Coordinates:
  * depth             (depth) int64 64B 50 100 150 200 250 300 350 400
  * is_day            (is_day) bool 2B False True
  * latitude          (latitude) float64 96B 46.5 47.5 48.5 ... 55.5 56.5 57.5
  * longitude         (longitude) float64 248B -157.5 -156.5 ... -128.5 -127.5
  * time              (time) datetime64[ns] 5kB 1995-09-24 ... 2020-08-31
Data variables:
    benthos           (time, is_day, latitude, longitude, depth) float64 30MB ...
    chaetognatha      (time, is_day, latitude, longitude, depth) float64 30MB ...
    crustacean        (time, is_day, latitude, longitude, depth) float64 30MB ...
    larvacean         (time, is_day, latitude, longitude, depth) float64 30MB ...
    mesh_size         (time, is_day, latitude, longitude, depth) float64 30MB ...
    others            (time, is_day, latitude, longitude, depth) float64 30MB ...
    small_gelatinous  (time, is_day, latitude, longitude, depth) float64 30MB ...
    thaliacea         (time, is_day, latitude, longitude, depth) float64 30MB ...
    total             (time, is_day, latitude, longitude, depth) float64 30MB ...
    volume_filtered   (time, is_day, latitude, longitude, depth) float64 30MB ...

In [3]:
results = xr.Dataset(
    {
        "day": papa_data["total"].sel(is_day=True).mean(["depth"]).drop("is_day"),
        "night": papa_data["total"].sel(is_day=False).mean(["depth"]).drop("is_day"),
    }
)
results

/var/folders/z_/8j3qx1mn0299kkpjgz9g53780000gq/T/ipykernel_29412/3158091827.py:3: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  "day": papa_data["total"].sel(is_day=True).mean(["depth"]).drop("is_day"),
/var/folders/z_/8j3qx1mn0299kkpjgz9g53780000gq/T/ipykernel_29412/3158091827.py:4: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  "night": papa_data["total"].sel(is_day=False).mean(["depth"]).drop("is_day"),


<xarray.Dataset> Size: 4MB
Dimensions:    (latitude: 12, longitude: 31, time: 623)
Coordinates:
  * latitude   (latitude) float64 96B 46.5 47.5 48.5 49.5 ... 55.5 56.5 57.5
  * longitude  (longitude) float64 248B -157.5 -156.5 -155.5 ... -128.5 -127.5
  * time       (time) datetime64[ns] 5kB 1995-09-24 1995-09-25 ... 2020-08-31
Data variables:
    day        (time, latitude, longitude) float64 2MB nan nan nan ... nan nan
    night      (time, latitude, longitude) float64 2MB nan nan nan ... nan nan

In [4]:
import plotly.express as px

day_df = (
    results["day"].mean(["latitude", "longitude"]).to_dataframe().reset_index().dropna()
)
night_df = (
    results["night"]
    .mean(["latitude", "longitude"])
    .to_dataframe()
    .reset_index()
    .dropna()
)


fig = px.line(
    title="Day and Night Series", labels={"value": "Abundance", "time": "Time"}
)

fig.add_scatter(x=day_df["time"], y=day_df["day"], mode="lines", name="Day")

fig.add_scatter(x=night_df["time"], y=night_df["night"], mode="lines", name="Night")

fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Wet weight mg/m3",
    legend_title="Day/Night",
    legend=dict(orientation="h"),
)

fig.show()


In [5]:
data_export = xr.Dataset(
    {
        "day": papa_data["total"].sel(is_day=True).mean(["depth"]).drop_vars("is_day"),
        "night": papa_data["total"]
        .sel(is_day=False)
        .mean(["depth"])
        .drop_vars("is_day"),
    }
)
data_export["day"].attrs = {
    "description": "Zooplankton dry weight by cube meter of sea water at day",
    "units": "mg/m3",
}
data_export["night"].attrs = {
    "description": "Zooplankton dry weight by cube meter of sea water at night",
    "units": "mg/m3",
}
data_export

<xarray.Dataset> Size: 4MB
Dimensions:    (latitude: 12, longitude: 31, time: 623)
Coordinates:
  * latitude   (latitude) float64 96B 46.5 47.5 48.5 49.5 ... 55.5 56.5 57.5
  * longitude  (longitude) float64 248B -157.5 -156.5 -155.5 ... -128.5 -127.5
  * time       (time) datetime64[ns] 5kB 1995-09-24 1995-09-25 ... 2020-08-31
Data variables:
    day        (time, latitude, longitude) float64 2MB nan nan nan ... nan nan
    night      (time, latitude, longitude) float64 2MB nan nan nan ... nan nan

In [6]:
data_export.to_zarr("Papa_zooplankton.zarr", mode="w")